# Eval

In [ ]:
!lang=ruby python run.py \
    --output_dir=./demo/python_model \
    --config_name=microsoft/codebert-base \
    --model_name_or_path=microsoft/codebert-base \
    --tokenizer_name=microsoft/codebert-base \
    --do_eval \
    --do_test \
    --train_data_file=codebase_ruby.jsonl \
    --eval_data_file=codebase_ruby.jsonl \
    --test_data_file=codebase_ruby.jsonl \
    --codebase_file=codebase.jsonl \
    --num_train_epochs 10 \
    --code_length 256 \
    --nl_length 128 \
    --train_batch_size 32 \
    --eval_batch_size 64 \
    --learning_rate 2e-5 \
    --seed 123456 2>&1| tee saved_models/ruby/test.log

02/21/2022 22:04:01 - INFO - __main__ -   device: cuda, n_gpu: 1
02/21/2022 22:04:08 - INFO - __main__ -   Training/evaluation parameters Namespace(train_data_file='codebase_ruby.jsonl', output_dir='./demo/python_model', eval_data_file='codebase_ruby.jsonl', test_data_file='codebase_ruby.jsonl', codebase_file='codebase.jsonl', model_name_or_path='microsoft/codebert-base', config_name='microsoft/codebert-base', tokenizer_name='microsoft/codebert-base', nl_length=128, code_length=256, do_train=False, do_eval=True, do_test=True, train_batch_size=32, eval_batch_size=64, learning_rate=2e-05, max_grad_norm=1.0, num_train_epochs=10, seed=123456, n_gpu=1, device='cuda:5')


In [1]:
%env CUDA_VISIBLE_DEVICES=0,2,3,4,5

env: CUDA_VISIBLE_DEVICES=0,2,3,4,5


In [2]:
from run import *

In [3]:
class Args():
    do_eval = True
    seed = 123456
    config_name = 'microsoft/codebert-base'
    tokenizer_name = 'microsoft/codebert-base'
    model_name_or_path = 'microsoft/codebert-base'
    do_train = False
    do_eval  = True
    do_test = False
    output_dir="./demo/python_model"
    eval_data_file="codebase_ruby.jsonl"
    code_length=256
    nl_length=128
    eval_batch_size=64
    codebase_file='codebase_ruby.jsonl'
    #set log
    
args = Args()
logging.basicConfig(format='%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                datefmt='%m/%d/%Y %H:%M:%S',level=logging.INFO )
#set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
args.n_gpu = torch.cuda.device_count()
args.device = 'cuda'
logger.info("device: %s, n_gpu: %s",device, args.n_gpu)

# Set seed
set_seed(args.seed)

#build model
config = RobertaConfig.from_pretrained(args.config_name if args.config_name else args.model_name_or_path)
tokenizer = RobertaTokenizer.from_pretrained(args.tokenizer_name)
model = RobertaModel.from_pretrained(args.model_name_or_path)
model=Model(model)
logger.info("Training/evaluation parameters %s", args)
model.to(args.device)

# Training
if args.do_train:
    train(args, model, tokenizer)

# Evaluation
results = {}
if args.do_eval:
    checkpoint_prefix = 'pytorch_model.bin'
    output_dir = os.path.join(args.output_dir, '{}'.format(checkpoint_prefix))
    model.load_state_dict(torch.load(output_dir),strict=False)
    model.to(args.device)
    

02/22/2022 21:40:15 - INFO - run -   device: cuda, n_gpu: 5
02/22/2022 21:40:22 - INFO - run -   Training/evaluation parameters <__main__.Args object at 0x7f2f3b4e97f0>


In [ ]:
# result=evaluate(args, model, tokenizer,args.eval_data_file)
# logger.info("***** Eval results *****")
# for key in sorted(result.keys()):
#     logger.info("  %s = %s", key, str(round(result[key],4)))

In [4]:
#def evaluate(args, model, tokenizer,file_name,eval_when_training=False):
file_name=args.eval_data_file
logger.info("***** Starting evaluation *****")
query_dataset = TextDataset(tokenizer, args, 'touf/test.jsonl')
query_sampler = SequentialSampler(query_dataset)
query_dataloader = DataLoader(query_dataset, sampler=query_sampler, batch_size=args.eval_batch_size,num_workers=4)

02/22/2022 21:40:24 - INFO - run -   ***** Starting evaluation *****


In [5]:
code_dataset = TextDataset(tokenizer, args, 'touf/codebase.jsonl')
code_sampler = SequentialSampler(code_dataset)
code_dataloader = DataLoader(code_dataset, sampler=code_sampler, batch_size=args.eval_batch_size,num_workers=4)
logger.info("***** Loaded data for evaluation *****")
# multi-gpu evaluate

02/22/2022 21:43:33 - INFO - run -   ***** Loaded data for evaluation *****


In [6]:
if args.n_gpu > 1:
    model = torch.nn.DataParallel(model)

# Eval!
logger.info("***** Running evaluation *****")
logger.info("  Num queries = %d", len(query_dataset))
logger.info("  Num codes = %d", len(code_dataset))
logger.info("  Batch size = %d", args.eval_batch_size)

02/22/2022 21:43:33 - INFO - run -   ***** Running evaluation *****
02/22/2022 21:43:33 - INFO - run -     Num queries = 52561
02/22/2022 21:43:33 - INFO - run -     Num codes = 183295
02/22/2022 21:43:33 - INFO - run -     Batch size = 64


In [7]:
model.eval()
pass

In [8]:
# code_vecs=[] ?
# nl_vecs=[]
nl_vecs_t= torch.empty((0,768), device=args.device)
code_vecs_t=torch.empty((0,768), device=args.device)
for batch in query_dataloader:  
    nl_inputs = batch[1].to(args.device)
    with torch.no_grad():
        nl_vec = model(nl_inputs=nl_inputs)
        # nl_vecs.append(nl_vec.cpu().numpy()) 
        nl_vecs_t = torch.cat((nl_vecs_t, nl_vec),dim=0)

In [9]:
for batch in code_dataloader:
    code_inputs = batch[0].to(args.device)    
    with torch.no_grad():
        code_vec= model(code_inputs=code_inputs)
        # code_vecs.append(code_vec.cpu().numpy())  
        code_vecs_t = torch.cat((code_vecs_t, code_vec),dim=0)

In [10]:
model.train()
pass

In [19]:
code_vecs_t = code_vecs_t.to('cuda:3')
nl_vecs_t = nl_vecs_t.to('cuda:3')

In [87]:
code_vecs_t.device

NameError: name 'code_vecs_t' is not defined

In [85]:
torch.cuda.empty_cache()

In [22]:
torch.save(code_vecs_t, 'code_vecs_t.pt')

In [23]:
torch.save(nl_vecs_t, 'nl_vecs_t.pt')

In [24]:
nl_chunks = torch.chunk(nl_vecs_t, 2, dim=0)

In [30]:
code_chunks = torch.chunk(code_vecs_t, 2, dim=0)

In [40]:
code_vecs_t2 = code_chunks[1].to('cuda:3')

In [41]:
nl_chunks_t2 = nl_chunks[1].to('cuda:3')

In [31]:
code_vecs_t1  = code_chunks[0].to('cuda:2')

In [32]:
nl_chunks_t1 = nl_chunks[0].to('cuda:2')

In [62]:
del nl_vecs_t
del code_vecs_t

NameError: name 'nl_vecs_t' is not defined

In [81]:
print(f"{code_vecs_t1}")
print(f"{nl_chunks_t1}")
print(f"{scores_t1}")
print(f"{code_vecs_t2}")
print(f"{nl_chunks_t2}")
print(f"{scores_t2}")

NameError: name 'code_vecs_t1' is not defined

In [63]:
del scores_t1
del scores_t2

In [79]:
# code_vecs=np.concatenate(code_vecs,0)
# nl_vecs=np.concatenate(nl_vecs,0)
# scores=np.matmul(nl_vecs,code_vecs.T)
# sort_ids=np.argsort(scores, axis=-1, kind='quicksort', order=None)[:,::-1]   

# print(f"Nl_vecs_t size: {nl_vecs_t.element_size() * nl_vecs_t.nelement()}")
# print(f"Nl_vecs_t elem size: {nl_vecs_t.element_size()}")
# print(f"Nl_vecs_t n elem: {nl_vecs_t.nelement()}")
# print(f"code_vecs_t size: {code_vecs_t.element_size() * code_vecs_t.nelement()}")
# print(f"nl_vecs shape: {nl_vecs_t.shape}")
# print(f"code_vecs_t shape: {code_vecs_t.shape}")
# scores_t1 = torch.matmul(nl_chunks_t1,code_vecs_t1.T)
scores_t2 = torch.matmul(nl_chunks_t2,code_vecs_t2.T)
print(scores_t1.device)
print(scores_t2.device)
# del nl_chunks_t1
# del nl_chunks_t2
# del code_vecs_t1
# del code_vecs_t2


NameError: name 'nl_chunks_t2' is not defined

In [82]:
print(torch.cuda.memory_summary())

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 1            |        cudaMalloc retries: 7         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |  707964 KB |    1704 MB |   16141 GB |   16140 GB |
|       from large pool |  707581 KB |    1704 MB |   16137 GB |   16136 GB |
|       from small pool |     383 KB |       2 MB |       4 GB |       4 GB |
|---------------------------------------------------------------------------|
| Active memory         |  707964 KB |    1704 MB |   16141 GB |   16140 GB |
|       from large pool |  707581 KB |    1704 MB |   16137 GB |

In [84]:
del scores_t1
del scores_t2

In [80]:
#scores_t.to(torch.device('cuda:3'))
#del nl_vecs_t
#del code_vecs_t
sort_ids_t1=torch.flip(torch.argsort(scores_t1, axis=-1),dims=(1,))
sort_ids_t2=torch.flip(torch.argsort(scores_t2, axis=-1),dims=(1,))
#del scores_t

RuntimeError: CUDA out of memory. Tried to allocate 17.95 GiB (GPU 3; 23.65 GiB total capacity; 19.29 GiB already allocated; 3.49 GiB free; 19.30 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [77]:
torch.cuda.empty_cache()

In [69]:
torch.save(scores_t2, 'scores_t2.pt')

In [86]:
%who

AdamW	 Args	 CrossEntropyLoss	 DataLoader	 Dataset	 InputFeatures	 MSELoss	 Model	 RandomSampler	 
RobertaConfig	 RobertaModel	 RobertaTokenizer	 SequentialSampler	 TensorDataset	 TextDataset	 WEIGHTS_NAME	 argparse	 args	 
batch	 checkpoint_prefix	 code_chunks	 code_dataloader	 code_dataset	 code_inputs	 code_sampler	 code_vec	 config	 
convert_examples_to_features	 device	 evaluate	 file_name	 get_linear_schedule_with_warmup	 json	 logger	 logging	 main	 
nl_inputs	 nl_vec	 np	 os	 output_dir	 pickle	 query_dataloader	 query_dataset	 query_sampler	 
random	 results	 set_seed	 tokenizer	 torch	 train	 


In [61]:
nl_urls=[]
code_urls=[]
for example in query_dataset.examples:
    nl_urls.append(example.url)

for example in code_dataset.examples:
    code_urls.append(example.url)

In [64]:
ranks=[]
for url, sort_id in zip(nl_urls,sort_ids_t.cpu().numpy()):
    rank=0
    find=False
    for idx in sort_id[:1000]: # if code_url matches nl_url, rank +1
        if find is False:
            rank+=1
        if code_urls[idx]==url:
            find=True
    if find:
        ranks.append(1/rank)
    else:
        ranks.append(0)

In [39]:
def get_mrr(indices, targets): #Mean Receiprocal Rank --> Average of rank of next item in the session.
    """
    Calculates the MRR score for the given predictions and targets
    Args:
        indices (Bxk): torch.LongTensor. top-k indices predicted by the model.
        targets (B): torch.LongTensor. actual target indices.
    Returns:
        mrr (float): the mrr score
    """
    tmp = targets.view(-1, 1)#.squeeze()
    # print(tmp)
    targets = tmp.expand_as(indices)
    hits = (targets == indices).nonzero()
    ranks = hits[:, -1] + 1
    ranks = ranks.float()
    rranks = torch.reciprocal(ranks)
    mrr = torch.sum(rranks).data / targets.size(0)
    return mrr

In [22]:
for example in query_dataset.examples:
    print(dir(example))
    print(example.code_ids)
    print(example.code_tokens)
    print(example.nl_ids)
    print(example.nl_tokens)
    print(example.url)
    print(example.uid)
    break

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'code_ids', 'code_tokens', 'nl_ids', 'nl_tokens', 'uid', 'url']
[0, 9232, 120, 1215, 34728, 1215, 1409, 1215, 13650, 36, 1651, 2156, 1546, 31723, 4839, 898, 5457, 38074, 1651, 646, 4832, 4135, 11655, 27779, 479, 349, 109, 1721, 1546, 1721, 114, 1546, 646, 321, 27779, 479, 159, 11173, 45994, 1546, 31723, 479, 159, 11173, 898, 5457, 120, 1215, 34728, 36, 1546, 646, 112, 27779, 4839, 253, 253, 898, 253, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [28]:
qdataset = {ex.uid:ex.url for ex in query_dataset.examples}

In [31]:
cdataset = {ex.uid:ex.url for ex in code_dataset.examples}

In [29]:
queries = [ex.uid for ex in query_dataset.examples]

In [83]:
len(query_dataset.examples)

52561

In [ ]:
nl_urls=[]
code_urls=[]
for example in query_dataset.examples:
    nl_urls.append(example.uid)

# for example in code_dataset.examples:
#     code_urls.append(example.uid)

ranks=[]
tranks=[]
ids_cpu = sort_ids_t
nl_cpu = torch.arange(len(query_dataset.examples), device=ids_cpu)
print(get_mrr(ids_cpu[:,:1000], nl_cpu))

In [48]:
ids_cpu.shape

torch.Size([4360, 4360])

In [52]:
for query_id, sort_id in zip(nl_urls,sort_ids_t.cpu().numpy()):
    # ranks.append())
    rank=0
    find=False
    for idx in sort_id[:1000]: # if code_url matches nl_url, rank +1
        if find is False:
            rank+=1
        if code_urls[idx] == query_id: #if code_urls[idx]==query_id:
            find=True
    if find:
        ranks.append(1/rank)
    else:
        ranks.append(0)
np.mean(ranks)

0.001716851114805125

In [158]:
nl_urls=[]
code_urls=[]
for example in query_dataset.examples:
    nl_ids.append(example.url)
for example in code_dataset.examples:
    code_urls.append(example.url)

sorted_ids = sort_ids_t.cpu()

mrrs = []
for targets, indices in zip(nl_ids,sorted_ids):
    targets = torch.tensor(targets)#.squeeze()
    print(targets.shape)
    print(indices.shape)
    tmp = torch.tensor(indices).view(-1, 1)#.squeeze()
    # print(tmp)
    targetss = tmp.expand_as(indices)
    hits = (targetss == indices).nonzero()
    ranks = hits[:, -1] + 1
    ranks = ranks.float()
    rranks = torch.reciprocal(ranks)
    mrr = torch.sum(rranks).data / targets.size(0)
    mrrs.append(mrr)
    # print(get_mrr(f[:128], ids))

torch.Size([128])
torch.Size([4360])


/tmp/ipykernel_43249/389760503.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tmp = torch.tensor(indices).view(-1, 1)#.squeeze()


RuntimeError: expand(torch.LongTensor{[4360, 1]}, size=[4360]): the number of sizes provided (1) must be greater or equal to the number of dimensions in the tensor (2)

In [65]:
result = {
    "eval_mrr":float(np.mean(mrrs))
}
result
#return result